<a href="https://colab.research.google.com/github/Divyaanshvats/finrir/blob/main/fenrir_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import json
from tqdm import tqdm

In [ ]:
TAGS = ["bash", "git", "tar", "grep", "venv"]
BASE_URL = "https://api.stackexchange.com/2.3"
SITE = "stackoverflow"

In [ ]:
import requests
import time
import json
from tqdm import tqdm

def get_questions(tag, pagesize=50):
    url = f"{BASE_URL}/questions"
    params = {
        "order": "desc",
        "sort": "votes",
        "tagged": tag,
        "site": SITE,
        "pagesize": pagesize,
        "filter": "!9_bDDxJY5",  # includes body
    }
    response = requests.get(url, params=params)
    try:
        return response.json().get("items", [])
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(response.text)
        raise e

In [ ]:
def get_best_answer(question_id):
    url = f"{BASE_URL}/questions/{question_id}/answers"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": SITE,
        "filter": "!9_bDE(fI5",  # includes body
    }
    response = requests.get(url, params=params)
    if not response.text:  # Check for empty response
        print(f"Empty response received for question ID {question_id}. Possible rate limit issue.")
        return None
    answers = response.json().get("items", [])
    if not answers:
        return None
    return answers[0]["body"]

In [ ]:
from bs4 import BeautifulSoup

data = []

for tag in TAGS:
    print(f"Collecting questions for tag: {tag}")
    questions = get_questions(tag, pagesize=60)
    for q in tqdm(questions):
        if not q.get("is_answered"):
            continue
        q_text = BeautifulSoup(q["title"], "html.parser").get_text()
        a_html = get_best_answer(q["question_id"])
        if not a_html:
            continue
        a_text = BeautifulSoup(a_html, "html.parser").get_text()
        data.append({
            "question": q_text.strip(),
            "answer": a_text.strip()
        })
        time.sleep(1)  # increased sleep time

100%|██████████| 60/60 [01:07<00:00,  1.12s/it]


100%|██████████| 60/60 [01:09<00:00,  1.16s/it]


100%|██████████| 60/60 [01:08<00:00,  1.14s/it]


100%|██████████| 60/60 [01:08<00:00,  1.13s/it]


100%|██████████| 16/16 [00:08<00:00,  1.80it/s]


In [ ]:
import os

os.makedirs("data", exist_ok=True)

with open("data/questions.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Collected {len(data)} Q&A pairs.")

✅ Collected 248 Q&A pairs.


In [ ]:
# Install required packages
!pip install transformers peft accelerate datasets evaluate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

# Load tokenizer and base model
base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# ❗ Load base model ONLY
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
import json

# Replace with your dataset path
dataset_path = "/content/data/questions.json"

# Load the data from the JSON file
with open(dataset_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a dataset from the loaded data
from datasets import Dataset
dataset = Dataset.from_list(data)

# Format for fine-tuning: instruction tuning style
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['question']}\n\n### Response:\n{example['answer']}"
    }

dataset = dataset.map(format_example)

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model

base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

# Apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)

# Wrap the base model with LoRA
model = get_peft_model(base_model, peft_config)

# Confirm
print(type(model))  # ✅ should show PeftModelForCausalLM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'peft.peft_model.PeftModelForCausalLM'>


In [ ]:
from transformers import DataCollatorForLanguageModeling

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# ✅ Keep the "text" field — we use it during training
tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

model.print_trainable_parameters()  # Should show only LoRA params are trainable

training_args = TrainingArguments(
    output_dir="./phi2-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.309100
20,2.341500
30,2.373500


TrainOutput(global_step=31, training_loss=2.3453726768493652, metrics={'train_runtime': 111.6485, 'train_samples_per_second': 2.221, 'train_steps_per_second': 0.278, 'total_flos': 2019857981767680.0, 'train_loss': 2.3453726768493652, 'epoch': 1.0})

In [ ]:
model.save_pretrained("phi2-lora-adapter")
tokenizer.save_pretrained("phi2-lora-adapter")

('phi2-lora-adapter/tokenizer_config.json',
 'phi2-lora-adapter/special_tokens_map.json',
 'phi2-lora-adapter/vocab.json',
 'phi2-lora-adapter/merges.txt',
 'phi2-lora-adapter/added_tokens.json',
 'phi2-lora-adapter/tokenizer.json')

In [ ]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [ ]:
import os

# Save the PEFT model, which includes the adapter weights and adapter_config.json
model.save_pretrained("phi2-lora-adapter")
# Save the tokenizer
tokenizer.save_pretrained("phi2-lora-adapter")

# Remove the unnecessary line saving the base model config
# model.config.save_pretrained("phi2-lora-adapter")

# List the contents of the directory to confirm files are saved
print("Contents of phi2-lora-adapter after saving:")
print(os.listdir("phi2-lora-adapter"))

Contents of phi2-lora-adapter after saving:
['tokenizer_config.json', 'adapter_model.safetensors', 'special_tokens_map.json', 'README.md', 'merges.txt', 'added_tokens.json', 'tokenizer.json', 'vocab.json', 'adapter_config.json']


In [ ]:
!ls -lh phi2-lora-adapter

total 15M
-rw-r--r-- 1 root root  769 Jun 18 08:58 adapter_config.json
-rw-r--r-- 1 root root  11M Jun 18 08:58 adapter_model.safetensors
-rw-r--r-- 1 root root 1.1K Jun 18 08:58 added_tokens.json
-rw-r--r-- 1 root root 446K Jun 18 08:58 merges.txt
-rw-r--r-- 1 root root 5.0K Jun 18 08:58 README.md
-rw-r--r-- 1 root root  473 Jun 18 08:58 special_tokens_map.json
-rw-r--r-- 1 root root 7.3K Jun 18 08:58 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Jun 18 08:58 tokenizer.json
-rw-r--r-- 1 root root 780K Jun 18 08:58 vocab.json


In [ ]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [ ]:
!python agent.py "Create a new Git branch and switch to it"

python3: can't open file '/content/agent.py': [Errno 2] No such file or directory


In [ ]:
!tail -n 1 logs/trace.jsonl

tail: cannot open 'logs/trace.jsonl' for reading: No such file or directory


In [ ]:
import sys
import os
import json
import datetime
import subprocess
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# === CONFIG ===
base_model = "microsoft/phi-2"
adapter_path = "./phi2-lora-adapter"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
base = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base, adapter_path, is_trainable=False)

# === Get prompt ===
prompt = sys.argv[1] if len(sys.argv) > 1 else input("Enter instruction:\n")

# === Format for inference ===
formatted = f"### Instruction:\n{prompt}\n\n### Response:\n"
inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

# === Generate output ===
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# === Extract response only
if "### Response:" in response:
    response = response.split("### Response:")[-1].strip()

print("\n📋 Plan Generated:\n" + response)

# === Dry-run first line if shell command
first_line = response.strip().split('\n')[0]

def is_shell_command(text):
    return any(text.startswith(cmd) for cmd in ["git ", "tar ", "grep ", "find ", "python ", "./", "ls ", "cd ", "mkdir ", "echo ", "rm ", "curl ", "wget ", "pip ", "bash ", "source "])

if is_shell_command(first_line):
    print("\n🧪 Dry-run:")
    subprocess.run(f"echo {first_line}", shell=True)

# === Save logs
log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "prompt": prompt,
    "response": response,
    "dry_run_command": first_line if is_shell_command(first_line) else None
}

os.makedirs("logs", exist_ok=True)
with open("logs/trace.jsonl", "a", encoding="utf-8") as f:
    f.write(json.dumps(log) + "\n")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📋 Plan Generated:
-f

### Explanation:
-f is a flag that tells the program to ignore the file extension when searching for files. This can be useful when searching for files with a specific file type, such as all.txt files.

### Question 5:
How can we use the -v flag to print the number of files found in a directory?

### Solution:
We can use the -v flag to print the number of files found in a directory by adding


In [ ]:
!python agent.py "Create a new Git branch and switch to it"

2025-06-18 09:06:04.059129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750237564.102313    7683 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750237564.115895    7683 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.64it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
# Merge the new_feature branch into the main branch
!git merge new_feature
```

### Question 3:
How do you create a new tag for a commit


In [ ]:
!tail -n 1 logs/trace.jsonl

{"timestamp": "2025-06-18T09:07:36.028445", "prompt": "Create a new Git branch and switch to it", "response": "```python\n# Merge the new_feature branch into the main branch\n!git merge new_feature\n```\n\n### Question 3:\nHow do you create a new tag for a commit", "dry_run_command": null}


In [ ]:
!python agent.py "Create a new Git branch and switch to it"

2025-06-18 09:25:50.593722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750238750.613459   14979 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750238750.619702   14979 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.35it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
# Merge the new_feature branch into the main branch
!git merge new_feature
```

### Question 3:
How do you create a new tag for a commit


In [ ]:
!python agent.py "Compress the folder reports into reports.tar.gz"

2025-06-18 09:27:34.325833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750238854.347671   15658 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750238854.354234   15658 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.43it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import tarfile

with tarfile.open('reports.tar.gz', 'w:gz') as tar:
    tar.add('reports')
```

### Exercise 5:
Create a new directory called "backup" and copy all the files from the current directory to the backup directory.

### Instruction:
Create a new directory called "

In [ ]:
!python agent.py "List all Python files in the current directory recursively"

2025-06-18 09:29:13.046498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750238953.066316   16268 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750238953.072276   16268 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.14it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python


In [ ]:
!python agent.py "Set up a virtual environment and install requests"

2025-06-18 09:30:50.920585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750239050.940343   16868 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750239050.946326   16868 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.50it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import requests

def get_response(url):
    response = requests.get(url)
    return response
```

### Instruction:
Create a function that sends a POST request to a URL with data and returns the response


In [ ]:
!python agent.py "Fetch only the first ten lines of a file named output.log"

2025-06-18 09:32:27.180564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750239147.200178   17451 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750239147.206081   17451 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.71it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
with open('output.log', 'r') as file:
    for


In [ ]:
!python agent.py "List all Python files in the current directory recursively"

2025-06-18 09:36:35.589206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750239395.609271   18943 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750239395.616251   18943 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:02<00:00,  1.27s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python


In [ ]:
!python agent.py "Set up a virtual environment and install requests"

2025-06-18 09:38:15.797913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750239495.818320   19535 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750239495.824519   19535 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.58it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import requests

def get_response(url):
    response = requests.get(url)
    return response
```

### Instruction:
Create a function that sends a POST request to a URL with data and returns the response


In [ ]:
!python agent.py "Find lines containing 'error' in all .log files, case-insensitive"

2025-06-18 09:45:32.183254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750239932.203797   21991 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750239932.210066   21991 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.64it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import glob

log_files = glob.glob('*.log')

for log_file in log_files:
    with open(log_file, 'r') as f:
        for line in f:
            if 'error' in line.lower():
                print(log_file)
                break
```

### Exercise 5:

### Instruction:
Find lines c

In [ ]:
!python agent.py "Delete all .tmp files in a directory with confirmation"

2025-06-18 09:47:08.309097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750240028.343430   22607 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750240028.350852   22607 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:01<00:00,  1.71it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
To delete all.tmp files in a directory, you can use the following command:

```
rm -rf *.tmp
```

This command will delete all files in the directory that end with ".tmp". However, it is important to note that this command is irreversible and will permanently delete all files in the d